In [2]:
import numpy as np
from scipy import sparse

import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1,max_mem_size = "12G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/alex/miniconda3/envs/h20/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5_hw1vgn
  JVM stdout: /tmp/tmp5_hw1vgn/h2o_alex_started_from_python.out
  JVM stderr: /tmp/tmp5_hw1vgn/h2o_alex_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_alex_83997t
H2O cluster total nodes:,1
H2O cluster free memory:,12 Gb
H2O cluster total cores:,6
H2O cluster allowed cores:,6
H2O cluster status:,"accepting new members, healthy"


In [3]:
X_test = sparse.load_npz("processed_data/raw_keyword_categorical_test_processed_20k_feat.npz")

X_test = h2o.H2OFrame(X_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
aml = h2o.load_model(path = "./saved_models/automl_001_bin/StackedEnsemble_BestOfFamily_AutoML_20200221_092127")

In [6]:
dir(aml)

['_ModelBase__generate_partial_plots',
 '_ModelBase__generate_user_splits',
 '_ModelBase__grabValues',
 '_ModelBase__plot_1dpdp',
 '_ModelBase__plot_2dpdp',
 '_ModelBase__predFor3D',
 '_ModelBase__setAxs1D',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_additional_used_columns',
 '_bc',
 '_bci',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_fillMultinomialDict',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_print_model_scoring_history',
 '_requires_tra

In [9]:
aml.end_time

1582321475512

In [10]:
predictions = aml.predict(X_test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [33]:
pd.Series(predictions.as_data_frame().iloc[:, 0])

0       0.560594
1       0.726838
2       0.938194
3       0.512097
4       0.954240
          ...   
3258    0.747774
3259    1.132983
3260    0.573864
3261    0.813188
3262    0.506105
Name: predict, Length: 3263, dtype: float64

In [26]:
test_id.shape

(3263,)

In [39]:
import pandas as pd
test_df = pd.read_csv("data/test.csv")
test_id = test_df['id']
test_predictions_df = pd.DataFrame([test_id, pd.Series(predictions.as_data_frame().iloc[:, 0])]).T
test_predictions_df.columns = ['id', 'target']

In [45]:
test_predictions_df['id'] = test_predictions_df['id'].astype('int')
test_predictions_df['target'] = test_predictions_df['target'].round()
test_predictions_df['target'] = test_predictions_df['target'].astype('int')

In [47]:
test_predictions_df.to_csv('test_preds_automl_001.csv', index=False)

In [48]:
# Shut down and release RAM
h2o.cluster().shutdown()

H2O session _sid_9f1c closed.
